# SAT NETs relevant to the 2025 "Pivot to Chile", Phase 2

January 2025

This notebook calculates NETs for the "Phase 2" baseline SATs, and some variations on that.
The baseline model has:
1) a HWP, large enough to enable f/1.45 optics,
2) horn sizes about the same as SO, which coupled with the same f/# as SO, means that we roughly match the spillover and edge
taper of SO.

This modeling uses flat bands, and a single number for each band's spillover.

NETs are calculated per detector, per wafer, and "per tube", where horn-horn correlations, yield, the number of wafers/tube and the optically good fraction of detectors in a tube are all taken into account.

In [1]:
%pwd

'c:\\Users\\Paul Grimes\\Documents\\GitHub\\bolo_calc_runs\\jbolo\\ChOpt_Report'

In [2]:

import sys
import os

sys.path.insert(0,'C:\\Users\\Paul Grimes\\Documents\\GitHub\\jbolo\\src')

os.environ['JBOLO_PATH'] = 'C:\\Users\\Paul Grimes\\Documents\\GitHub\\jbolo'

In [3]:
print(sys.path)

['C:\\Users\\Paul Grimes\\Documents\\GitHub\\jbolo\\src', 'c:\\Users\\pgrimes\\miniforge\\envs\\jbolo\\python313.zip', 'c:\\Users\\pgrimes\\miniforge\\envs\\jbolo\\DLLs', 'c:\\Users\\pgrimes\\miniforge\\envs\\jbolo\\Lib', 'C:\\Users\\Paul Grimes\\miniforge\\envs\\jbolo', 'c:\\Users\\pgrimes\\miniforge\\envs\\jbolo', '', 'c:\\Users\\pgrimes\\miniforge\\envs\\jbolo\\Lib\\site-packages', 'c:\\Users\\pgrimes\\miniforge\\envs\\jbolo\\Lib\\site-packages\\win32', 'c:\\Users\\pgrimes\\miniforge\\envs\\jbolo\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\pgrimes\\miniforge\\envs\\jbolo\\Lib\\site-packages\\Pythonwin']


In [4]:
import os

import numpy as np
import matplotlib.pyplot as plt
import yaml
from jbolo_funcs import *

%matplotlib inline
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.figsize'] = [8, 4]

In [5]:
%cd "c:\\Users\\Paul Grimes\\Documents\\GitHub\\bolo_calc_runs\\jbolo\\ChOpt_Report"
# List what yaml files you have so you can choose one for the next line.
%ls -lt1 yamls 

c:\Users\Paul Grimes\Documents\GitHub\bolo_calc_runs\jbolo\ChOpt_Report
 Volume in drive C is OS
 Volume Serial Number is 781B-3C01

 Directory of c:\Users\Paul Grimes\Documents\GitHub\bolo_calc_runs\jbolo\ChOpt_Report


 Directory of c:\Users\Paul Grimes\Documents\GitHub\bolo_calc_runs\jbolo\ChOpt_Report\yamls

03/22/2025  14:33    <DIR>          .
03/22/2025  14:33    <DIR>          ..
03/22/2025  15:07             8,909 SAT_2025_AoAalt1.yaml
03/22/2025  15:07             8,951 SAT_2025_PoleAlt1.yaml
03/22/2025  15:07             8,910 SAT_2025_PoleAlt1_273K.yaml
               3 File(s)         26,770 bytes
               2 Dir(s)  375,975,342,080 bytes free


File Not Found


# Compare Baffle and Window Temps at Pole
Two options here:
- CMB-S4 baseline, 273K baffle and window
- BICEP3 estimated radiation temperatures - 240K forebaffle, 260K window

In [36]:
# Dictionary of models to use, which includes yaml file for now, but will have results in it later.
SAT_models = {}

# Thin (1.5mm) window, otherwise as for Phase 2
SAT_models['Chile Phase 2 Single MF']={  
    'yaml':'SAT_HWP_aggr_MF.yaml'
}

# Phase 2, 20mm HDPE window
SAT_models['Chile Phase 2']={  
    'yaml':'SAT_HWP_aggr_20241121_phase2.yaml'
}

# As above, no half-wave plate
SAT_models['Chile P2 No HWP']={  
    'yaml':'SAT_aggr_20241121_phase2.yaml'
}

# No HWP, 30 deg FOV
SAT_models['Chile P2 No HWP 30 deg FOV']={  
    'yaml':'SAT_aggr_20241121_30degFOV.yaml'
}

# Pole, aggressive 35 deg FOV
SAT_models['Pole P2 No HWP']={  
    'yaml':'Pole_SAT_aggr_20241121_phase2.yaml'
}

# Pole, aggressive 30 deg FOV
SAT_models['Pole P2 No HWP 30 deg FOV']={  
    'yaml':'Pole_SAT_aggr_20241121_30degFOV.yaml'
}

# Pole, original
SAT_models['Pole Orig']={  
    'yaml':'Pole_SAT_orig_20241121_30degFOV.yaml'
}

We're going to do this a very simple way, where we explicitly run each sim and plot it before moving on to the next.


In [37]:
models = list(SAT_models.keys())

uhmwpe_tandelta = 1.2e-4
dyneema_tandelta = 2.4e-4

sims={}
# Run each model
for model in models:
    # Set up
    yamlfile = os.path.join('yamls', SAT_models[model]['yaml'])
    sim = yaml.safe_load(open(yamlfile))
    SAT_models[model]['comment'] = 'NETs are in uKrtsec; powers are in pW'
    SAT_models[model]['name'] = model #sim['version']['name']
    SAT_models[model]['date'] = sim['version']['date']
    SAT_models[model]['site'] = sim['sources']['atmosphere']['site']
    SAT_models[model]['pwv'] = sim['sources']['atmosphere']['pwv']
    SAT_models[model]['elev'] = sim['sources']['atmosphere']['elevation']
    SAT_models[model]['wafers'] = sim['bolo_config']['num_wafers_per_tube']
    SAT_models[model]['optically_good_fraction'] = sim['bolo_config']['optically_good_det_fraction']
    
    # Run
    run_optics(sim)
    run_bolos(sim)
    #print_optics(sim,'MF_1_1')
 
    # Pack outputs into the models dictionary
    SAT_models[model]['channels']=sim['outputs'].keys()
    ii = 0
    for ch in SAT_models[model]['channels']:
        SAT_models[model][ch]={}
        net1det = 1e6*sim['outputs'][ch]['NET_C_total']  # use the version with horn-horn correlations
        ndet = sim['channels'][ch]['num_det_per_wafer']  # detectors per wafer
        net_wafer = net1det/np.sqrt(ndet*sim['bolo_config']['yield'])  #Take yield into account, usually 80%
        net_tube = net_wafer/np.sqrt(SAT_models[model]['wafers']*SAT_models[model]['optically_good_fraction'])  #Take yield into account, usually 80%
        ms = 1/net_tube**2
        SAT_models[model][ch]['P_opt'] = 1e12*sim['outputs'][ch]['P_opt']   #convert to pW from W
        SAT_models[model][ch]['NET_wafer'] = net_wafer   #convert to uKrtsec from Krtsec
        SAT_models[model][ch]['NET_tube'] = net_tube
        SAT_models[model][ch]['mapping_speed'] = ms
        SAT_models[model][ch]['NET_C'] = net1det
        SAT_models[model][ch]['corr_factor'] = sim['outputs'][ch]['corr_factor']
        SAT_models[model][ch]['NET_NC']=1e6*sim['outputs'][ch]['NET_NC_total']
        chnum = sim['channels'][ch]['chnum']
        SAT_models[model][ch]['Lyot_abs'] = sim['optical_elements']['lyot']['absorption'][chnum]
        ii += 1
        
# Print a table for each model
for model in models:
    print(SAT_models[model]['name'], end=', \t')
    print(SAT_models[model]['site'], end=', ')
    print('\tpwv =',SAT_models[model]['pwv'],'u', end=', ')
    print('\telev =',SAT_models[model]['elev'])
    #print('            NET_NC   corr_fact NET_C    NET_wafer  NET_tube map_speed  ms_ratio     P_opt')
    print('         ,\t NET_NC,\t ms_ratio,\t P_opt')
    for ch in SAT_models[model]['channels']:
        try:
            msratio = SAT_models[model][ch]['mapping_speed']/SAT_models['Pole Orig'][ch]['mapping_speed']
        except KeyError:
            ch_num = ch.split("_")[-1]
            msratio = 2*SAT_models[model][ch]['mapping_speed']/(SAT_models['Pole Orig'][f'MF_1_{ch_num}']['mapping_speed'] + SAT_models['Pole Orig'][f'MF_2_{ch_num}']['mapping_speed'])
        print(ch.ljust(10),end=',\t')
#        print('{0:8.1f}  {1:8.3f} {2:8.1f}  {3:8.1f} {4:8.2f}  {5:10.2e}  {6:8.2f} {7:10.2f}'.format(SAT_models[model][ch]['NET_NC'], 
#                                                                                           SAT_models[model][ch]['corr_factor'],
#                                                                                           SAT_models[model][ch]['NET_C'],
#                                                                                           SAT_models[model][ch]['NET_wafer'], 
#                                                                                           SAT_models[model][ch]['NET_tube'],
#                                                                                           SAT_models[model][ch]['mapping_speed'],
#                                                                                           msratio,
#                                                                                           SAT_models[model][ch]['P_opt']))
        print('{0:8.1f},\t{1:8.2f},\t{2:10.2f}'.format(SAT_models[model][ch]['NET_NC'], 
                                                    msratio,
                                                    SAT_models[model][ch]['P_opt']))

    print()
    
    

Chile Phase 2 Single MF, 	Atacama, 	pwv = 993 u, 	elev = 50
         ,	 NET_NC,	 ms_ratio,	 P_opt
MF_1      ,	   239.2,	    3.64,	      1.75
MF_2      ,	   293.8,	    2.90,	      4.68

Chile Phase 2, 	Atacama, 	pwv = 993 u, 	elev = 50
         ,	 NET_NC,	 ms_ratio,	 P_opt
LF_1      ,	   214.1,	    2.31,	      0.28
LF_2      ,	   147.5,	    4.32,	      1.28
MF_1_1    ,	   268.3,	    3.44,	      1.39
MF_2_1    ,	   245.1,	    2.82,	      1.57
MF_1_2    ,	   305.3,	    2.67,	      3.80
MF_2_2    ,	   343.2,	    2.13,	      4.57
HF_1      ,	   719.6,	    0.81,	     11.23
HF_2      ,	  1816.9,	    0.76,	     14.66

Chile P2 No HWP, 	Atacama, 	pwv = 993 u, 	elev = 50
         ,	 NET_NC,	 ms_ratio,	 P_opt
LF_1      ,	   211.0,	    2.38,	      0.28
LF_2      ,	   145.3,	    4.45,	      1.30
MF_1_1    ,	   262.5,	    3.59,	      1.40
MF_2_1    ,	   238.9,	    2.97,	      1.58
MF_1_2    ,	   295.9,	    2.84,	      3.80
MF_2_2    ,	   332.8,	    2.27,	      4.58
HF_1      ,	   697.2,	    0.87,	  

In [ ]:
%pwd

'c:\\Users\\Paul Grimes\\Documents\\GitHub\\bolo_calc_runs\\jbolo\\ChOpt_Report'